## Demande de viz complémentaire axe 2.1 

En réponse au commentaire de Silvia du 21/0( sur doc étape 3 : "je verrai bien une carte avec les données Navigo de 1787, qui montre de quels ports français [pointcall OUT ou IN-OUT] sortent les navires de La Rochelle [homeport] et leur destination suivante avec une mise en valeur d'une manière ou d'une autre du tonnage (par ex: possibilité de cliquer pour voir le Nombre OU le tonnage total cumulé"

#### Données utilisées : 
pointcalls navigo : 
year = 1787
homeport = 'La Rochelle'
pointcall_action = ['Out', 'In-Out', 'In-out'] 

#### Construction de la carte : 
Production d'un jeu de données mettant en correspondance les points de départ ou d'étape de ces navires (attribut 'pointcall' des pointcalls 'Out' et 'In-Out'), avec leur destination (attribut 'pointcall' des pointcalls 'In' ayant le même source_doc_id).

Si le même trajet est réalisé plusieurs fois, on aggrège par nombre de trajets, ou tonnage cumulé, et on adapte la largeur des flux sur la carte.

In [1]:
# imports
import json 
from poitousprint import Portic, get_pointcalls_commodity_purposes_as_toflit_product

portic_client = Portic()

In [6]:
# collecte des départs et points d'étape
pointcalls_departures_la_rochelle_homeport = portic_client.get_pointcalls(
    year = 1787, 
    homeport = 'La Rochelle', # après vérification, tous les pointcalls 'Out' de 1789 qui concernent les ports rattachés au bureau des Fermes de Marennes ont bien l'attribut ferme_bureau à 'Marennes'
    pointcall_action = ['Out', 'In-Out', 'In-out'] # peut être encore des soucis de casse dans l'API 
)

print("nombre de pointcalls Out ou In-Out (départs et étapes) trouvés pour desnavires du port de La Rochelle:", len(pointcalls_departures_la_rochelle_homeport))

nombre de pointcalls Out ou In-Out (départs et étapes) trouvés pour desnavires du port de La Rochelle: 360


In [15]:
# collecte de leurs source_doc_id
pointcalls_departures_source_doc_ids = set()

for p in pointcalls_departures_la_rochelle_homeport:
    pointcalls_departures_source_doc_ids.add(p['source_doc_id'])
    
print("Nombre de source_doc_id trouvés : ", len(pointcalls_departures_source_doc_ids))

00154072
00149640
00154055
00149450
00149634
00154054
00154060
00154050
00154052
00154059
00154070
00154086
00154082
Nombre de source_doc_id trouvés :  347


In [10]:
# retrouver les destinations : 1ere étape de collecte des pointcalls In
pointcalls_in_la_rochelle_homeport = portic_client.get_pointcalls(
    start_year = 1787,
    end_year = 1788, # on considère que le voyage ne dure pas plus d'un an
    pointcall_action = ['In', 'in'] # peut être encore des soucis de casse dans l'API 
)

print("Nombre de pointcalls In trouvés entre 1787 et 1788 :", len(pointcalls_in_la_rochelle_homeport))

Nombre de pointcalls In trouvés entre 1787 et 1788 : 35750


In [12]:
# retrouver les destinations : 2 etape de filtrage des pointcalls In considérés comme destination des Out sortant de Marennes avec du sel (même pkid)
pointcalls_destinations_la_rochelle_homeport = []

for p in pointcalls_in_la_rochelle_homeport:
    if (str(p['source_doc_id']) in pointcalls_departures_source_doc_ids):
        pointcalls_destinations_la_rochelle_homeport.append(p)

print("Nombre de destinations pertinentes trouvées : ", len(pointcalls_destinations_la_rochelle_homeport))

Nombre de destinations pertinentes trouvées :  344


In [13]:
import pandas as pds

# Load an empty map
from keplergl import KeplerGl

kepler_base_conf = {
    'version': 'v1',
    'config': {
        # centering the map on the region
        'mapState': {
            'latitude': 45.6876849,
            'longitude': -1.15,
            'zoom': 5.2
        }
    }
}

In [14]:
flows_la_rochelle_homeport_1787 = {}
## but de reconstruire dicts qui mettent en relation les trajets qui ont le même source doc id mais je ne sais comment gérer les etapes
{
    'source_doc_id':
    { 'lat_dep':
      'lon_dep':
      'lat-step': # après vérification on a au max 1 escale dans les données qu'on utilise
      'lon_step': 
      'lat_dest':
      'lon_dest':
      'tonnage':
    }
}

for p in pointcalls_departures_la_rochelle_homeport:
    port = p['pointcall']
    # french_flag = p['ship_flag_standardized_fr'] in ['français'] 
    tonnage = int(p['tonnage']) if p['tonnage'] is not None else 0
    
    if p['source_doc_id'] not in flows_la_rochelle_homeport_1787:
        lat = float(p['latitude']) if p['latitude'] is not None else None
        lon = float(p['longitude']) if p['longitude'] is not None else None
        if p['pointcall_action']=='Out':
        flows_la_rochelle_homeport_1787[p['source_doc_id']] = {
                'lat_dep':lat,
                'lon_dep':lon,
                'latitude_dest': lat,
                'longitude_dest': lon,
                'sous_etat': p['substate_1789_fr'],
                'etat': p['state_1789_fr'],
                'tonnage':tonnage,
            }
        else:
            ports_dest_french_flag[port]['nb_pointcalls'] += 1
            ports_dest_french_flag[port]['tonnages'] += tonnage
    else:
        if port not in ports_dest_ext_flag:
            lat = float(p['latitude']) if p['latitude'] is not None else None
            lon = float(p['longitude']) if p['longitude'] is not None else None
            ports_dest_ext_flag[port] = {
                'latitude_dep':45.81666,
                'longitude_dep':-1.116667, 
                'nb_pointcalls': 1,
                'latitude_dest': lat,
                'longitude_dest': lon,
                'sous_etat': p['substate_1789_fr'],
                'etat': p['state_1789_fr'],
                'tonnages':tonnage,
            }
        else:
            ports_dest_ext_flag[port]['nb_pointcalls'] += 1
            ports_dest_ext_flag[port]['tonnages'] += tonnage
            
ports_dest_french_flag = [{'port': port, **vals} for port, vals in ports_dest_french_flag.items()]
ports_dest_ext_flag = [{'port': port, **vals} for port, vals in ports_dest_ext_flag.items()]

# print("french flags : ", ports_dest_french_flag[0:10])
# print("\n\next flags : ", ports_dest_ext_flag[0:20])

with open('assets/results_french.json', 'w') as fp1:
    json.dump(ports_dest_french_flag, fp1)
with open('assets/results_ext.json', 'w') as fp2:
    json.dump(ports_dest_ext_flag, fp2)

NameError: name 'final_pointcalls_in' is not defined